In [1]:
import pandas as pd
import numpy as np
from imdb import Cinemagoer
import matplotlib.pyplot as plt
import datetime
from collections import Counter

from cleaning_funcs import load_data

ia = Cinemagoer()

%matplotlib inline

#IMDb Docs: https://imdbpy.readthedocs.io/en/latest/index.html

In [2]:
data = load_data('movie_list_current.csv')
data = data[['Month', 'Pick', 'IMDb ID', 'Movie', 'Year', 'IMDb Score', 'Steven', 'Jon', 'Farzad', 'Average', 'Notes']]
print("Number of Movies:", data.shape[0])
data = data[data.Steven.notnull()]
data.head(3)

Number of Movies: 93


,Month,Pick,IMDb ID,Movie,Year,IMDb Score,Steven,Jon,Farzad,Average,Notes
0,3/2019,Farzad,947798,Black Swan,2010,8.0,8.0,9.0,8.0,8.3,Inagural movie!
1,4/2019,Jon,156887,Perfect Blue,1997,8.0,9.0,9.0,7.0,8.3,"Black Swan, but make it anime."
2,5/2019,Steven,311113,Master and Commander: The Far Side of the World,2003,7.4,9.5,8.0,8.0,8.5,https://www.youtube.com/watch?v=mq1YthGFjRI


In [5]:
# subset = data[0:10]
# test_movie = ia.get_movie(subset.loc[0]['IMDb ID'])
# test_movie.get('title')

'Black Swan'

In [75]:
subset = data[0:3]

for index, row in subset.iterrows():
    temp = pd.DataFrame()

    movie = ia.get_movie(row['IMDb ID'])
    
    title = movie.get('title')
    year = movie.get('year')
    # movie_data['imdbid'] = movie.get('imdbid')
    # movie_data['directors'] = movie.get('directors') #need to parse this before we can use it
    rating = movie.get('rating')
    # movie_data['genres'] = movie.get('genres')
    # movie_data['runtimes'] = movie.get('runtimes')
    # movie_data['budget'] = movie.get('budget')
    # movie_data['gross'] = movie.get('gross')
    # movie_data['plot'] = movie.get('plot')
    # movie_data['synopsis'] = movie.get('synopsis')

    temp.append(title, year, rating)

pd.concat(subset, temp)

/var/folders/8m/_5jnf1912wg4yn1spmnkt87c0000gn/T/ipykernel_37186/3948858551.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp.append(title, year, rating)


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

{'title': ['Master and Commander: The Far Side of the World'],
 'year': [2003],
 'score': [7.4]}